In [42]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from scipy import stats
import numpy as np

In [43]:

df = pd.read_csv("Fish.csv")
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [44]:
df_num = df.select_dtypes(include=[np.number]).dropna()


y = df_num["Weight"]

X_cols = ["Length1","Length2","Length3","Height","Width"]
X = df_num[X_cols]

In [54]:
X = sm.add_constant(X)

# модель
model1 = sm.OLS(y, X).fit()

print(model1.summary())

mse1 = np.mean(model1.resid**2)
print("MSE (модель 1):", round(mse1, 4))

sy = np.std(y, ddof=1)

sx1 = df_num[["Length1","Length2","Length3","Height","Width"]].std(ddof=1).values
betas1 = model1.params[1:] * (sx1 / sy)
eta1 = model1.rsquared - np.sum(betas1**2)
print("η_system (модель 1):", round(eta1, 4))

# VIF
vif1 = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("\nVIF:")
for name, val in zip(X.columns, vif1):
    print(name, round(val, 2))

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     236.2
Date:                Thu, 02 Oct 2025   Prob (F-statistic):           4.95e-70
Time:                        21:09:57   Log-Likelihood:                -987.96
No. Observations:                 159   AIC:                             1988.
Df Residuals:                     153   BIC:                             2006.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -499.5870     29.572    -16.894      0.0

In [46]:
# матрица корреляций
corr_matrix = df_num[["Weight"]+X_cols].corr()

# значимость факторов (p-values)
pvals = coef_table1["P>|t|"].iloc[1:]
selected = pvals[pvals < 0.05].index.tolist()

if len(selected) < 2:  # на случай если <2 значимых
    selected = pvals.sort_values().index[:2].tolist()

corr_matrix, pd.DataFrame({"p-value":pvals,"selected":pvals.index.isin(selected)})

(           Weight   Length1   Length2   Length3    Height     Width
 Weight   1.000000  0.915712  0.918618  0.923044  0.724345  0.886507
 Length1  0.915712  1.000000  0.999517  0.992031  0.625378  0.867050
 Length2  0.918618  0.999517  1.000000  0.994103  0.640441  0.873547
 Length3  0.923044  0.992031  0.994103  1.000000  0.703409  0.878520
 Height   0.724345  0.625378  0.640441  0.703409  1.000000  0.792881
 Width    0.886507  0.867050  0.873547  0.878520  0.792881  1.000000,
           p-value  selected
 Length1  0.123019     False
 Length2  0.876005     False
 Length3  0.096431      True
 Height   0.001458      True
 Width    0.271689     False)

In [47]:
print("\nКорреляции y~x:")
print(df_num[X_cols + ['Weight']].corr()['Weight'].drop('Weight'))

print("\nКорреляции между факторами:")
print(df_num[X_cols].corr())

print("\nP-values по факторам:")
for name, val in model1.pvalues.items():
    print(f"{name:8} : {val:.4f}")

selected = [c for c in X_cols if model1.pvalues[c] < 0.05]
for c in df_num[X_cols].corrwith(df_num['Weight']).abs().sort_values(ascending=False).index:
    if len(selected) >= 2: break
    if c not in selected: selected.append(c)

print("\nВыбранные факторы:", selected)


Корреляции y~x:
Length1    0.915712
Length2    0.918618
Length3    0.923044
Height     0.724345
Width      0.886507
Name: Weight, dtype: float64

Корреляции между факторами:
          Length1   Length2   Length3    Height     Width
Length1  1.000000  0.999517  0.992031  0.625378  0.867050
Length2  0.999517  1.000000  0.994103  0.640441  0.873547
Length3  0.992031  0.994103  1.000000  0.703409  0.878520
Height   0.625378  0.640441  0.703409  1.000000  0.792881
Width    0.867050  0.873547  0.878520  0.792881  1.000000

P-values по факторам:
const    : 0.0000
Length1  : 0.1230
Length2  : 0.8760
Length3  : 0.0964
Height   : 0.0015
Width    : 0.2717

Выбранные факторы: ['Height', 'Length3']


In [55]:
X2 = df_num[selected]
X2 = sm.add_constant(X2)

# модель 2
model2 = sm.OLS(y, X2).fit()

print(model2.summary())

mse2 = np.mean(model2.resid**2)
print("MSE (модель 2):", round(mse2, 4))

sx2 = df_num[["Height","Length3"]].std(ddof=1).values
betas2 = model2.params[1:] * (sx2 / sy)
eta2 = model2.rsquared - np.sum(betas2**2)

print("η_system (модель 2):", round(eta2, 4))

# VIF для новой модели
vif2 = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
print("\nVIF (новая модель):")
for name, val in zip(X2.columns, vif2):
    print(name, round(val, 2))

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     492.0
Date:                Thu, 02 Oct 2025   Prob (F-statistic):           4.20e-68
Time:                        21:10:23   Log-Likelihood:                -1002.0
No. Observations:                 159   AIC:                             2010.
Df Residuals:                     156   BIC:                             2019.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -501.0970     30.559    -16.398      0.0

In [49]:
f_test = model1.compare_f_test(model2)

print("F-тест (полная vs сокращённая модель):")
print(f"F-статистика = {f_test[0]:.4f}")
print(f"p-value      = {f_test[1]:.4f}")
print(f"df_diff      = {int(f_test[2])}")

F-тест (полная vs сокращённая модель):
F-статистика = 9.8357
p-value      = 0.0000
df_diff      = 3


In [56]:
res = model2.resid
n = len(res)

# число поворотных точек
K = 0
for i in range(1, n-1):
    if (res[i] > res[i-1] and res[i] > res[i+1]) or (res[i] < res[i-1] and res[i] < res[i+1]):
        K += 1

EK = 2*(n-2)/3
VarK = (16*n - 29)/90
zK = (K - EK)/np.sqrt(VarK)
pK = 2*(1 - stats.norm.cdf(abs(zK)))

print("Поворотные точки (модель 2):")
print("наблюдаемое K =", K, ", ожидаемое E[K] =", round(EK, 2))
print("z =", round(zK, 3), ", p =", round(pK, 4))

Поворотные точки (модель 2):
наблюдаемое K = 93 , ожидаемое E[K] = 104.67
z = -2.207 , p = 0.0273


In [50]:
resid = model2.resid

# 1. Сумма остатков
print("Сумма остатков:", round(resid.sum(), 6))

# 2. Критерий Дарбина–Уотсона
dw = durbin_watson(resid)
print("Критерий Дарбина–Уотсона:", round(dw, 3))

# 3. Нормальность: асимметрия и эксцесс
asim = stats.skew(resid)
excess = stats.kurtosis(resid)
print("Асимметрия остатков:", round(asim, 3))
print("Эксцесс остатков:", round(excess, 3))

Сумма остатков: -0.0
Критерий Дарбина–Уотсона: 0.425
Асимметрия остатков: 0.35
Эксцесс остатков: 0.15
